## Final Project Submission

Please fill out:
* Student name: Amani Wanene
* Student pace:  full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


1. Import relevant resources

In [4]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd
import os
import sqlite3
import matplotlib.pyplot as plt
import numpy as np


2. Import the datasets.

In [53]:
bom_df = pd.read_csv('zippedData/bom.movie_gross.csv.gz',encoding = 'latin1')
movie_info_df = pd.read_csv('zippedData/rt.movie_info.tsv.gz',delimiter = '\t')
reviews_df = pd.read_csv('zippedData/rt.reviews.tsv.gz',delimiter = '\t',encoding = 'latin1')
tmdb_df = pd.read_csv('zippedData/tmdb.movies.csv.gz')
movie_budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')


In [2]:


import zipfile

# Extract IMDb SQL .db file
with zipfile.ZipFile('./zippedData/im.db.zip') as zipObj:
    # Extract all contents of .zip file into current directory
    zipObj.extractall(path='./zippedData/')
    
# Create connection to IMDb DB


In [13]:
imdb_conn = sqlite3.connect('zippedData/im.db')


In [42]:
imdb_df = pd.read_sql("""SELECT * FROM movie_basics
 JOIN movie_ratings
USING(movie_id)
JOIN movie_akas
USING(movie_id)
""",imdb_conn)

3.View the datasets

In [68]:
print(bom_df.shape)
bom_df.isna().sum()




(3387, 5)


title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [69]:
print(movie_info_df.shape)
movie_info_df.isna().sum()

(1560, 12)


id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

In [71]:
print(reviews_df.shape)
reviews_df.isna().sum()

(54432, 8)


id                0
review         5563
rating        13517
fresh             0
critic         2722
top_critic        0
publisher       309
date              0
dtype: int64

In [65]:
tmdb_df.isna().sum()

Unnamed: 0           0
genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

In [66]:
movie_budgets_df.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [70]:
print(imdb_df.shape)
imdb_df.isna().sum()

(261806, 15)


movie_id                  0
primary_title             0
original_title            0
start_year                0
runtime_minutes       11253
genres                 1185
averagerating             0
numvotes                  0
ordering                  0
title                     0
region                43465
language             224726
types                108538
attributes           248882
is_original_title         0
dtype: int64

4. Clean Data

A)